# **Aspect-based Term Extraction using BERTs**

##**Dataset**

In [ ]:
!pip install -q datasets==3.2.0

In [2]:
from datasets import load_dataset

ds = load_dataset("thainq107/abte-restaurants")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/454 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/183k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/61.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3602 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1119 [00:00<?, ? examples/s]

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['Tokens', 'Tags', 'Polarities'],
        num_rows: 3602
    })
    test: Dataset({
        features: ['Tokens', 'Tags', 'Polarities'],
        num_rows: 1119
    })
})

In [4]:
ds['train'][0]

{'Tokens': ['But', 'the', 'staff', 'was', 'so', 'horrible', 'to', 'us', '.'],
 'Tags': ['0', '0', '1', '0', '0', '0', '0', '0', '0'],
 'Polarities': ['-1', '-1', '0', '-1', '-1', '-1', '-1', '-1', '-1']}

## **Tokenizer**

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = []
    labels = []
    for tokens, tags in zip(examples['Tokens'], examples['Tags']):

        bert_tokens = []
        bert_tags = []
        for i in range(len(tokens)):
            t = tokenizer.tokenize(tokens[i])
            bert_tokens += t
            bert_tags += [int(tags[i])]*len(t)

        bert_ids = tokenizer.convert_tokens_to_ids(bert_tokens)

        tokenized_inputs.append(bert_ids)
        labels.append(bert_tags)

    return {
            'input_ids': tokenized_inputs,
            'labels': labels
        }

In [7]:
preprocessed_ds = ds.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3602 [00:00<?, ? examples/s]

Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

In [13]:
preprocessed_ds.column_names

{'train': ['Tokens', 'Tags', 'Polarities', 'input_ids', 'labels'],
 'test': ['Tokens', 'Tags', 'Polarities', 'input_ids', 'labels']}

In [8]:
preprocessed_ds['train'][0]

{'Tokens': ['But', 'the', 'staff', 'was', 'so', 'horrible', 'to', 'us', '.'],
 'Tags': ['0', '0', '1', '0', '0', '0', '0', '0', '0'],
 'Polarities': ['-1', '-1', '0', '-1', '-1', '-1', '-1', '-1', '-1'],
 'input_ids': [2021, 1996, 3095, 2001, 2061, 9202, 2000, 2149, 1012],
 'labels': [0, 0, 1, 0, 0, 0, 0, 0, 0]}

In [9]:
len(tokenizer)

30522

## **Data Collator**

In [10]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## **Evaluate**

In [11]:
!pip install -q seqeval==1.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [14]:
id2label = {
    0: "O",
    1: "B-Term",
    2: "I-Term"
}
label2id = {
    "O": 0,
    "B-Term": 1,
    "I-Term": 2
}

In [15]:
import numpy as np
from seqeval.metrics import accuracy_score, f1_score

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = f1_score(true_predictions, true_labels)
    return {"F1-score": results}

## **Model**

In [16]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased",
    num_labels=3, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
   

## **Training**

In [18]:
# Disable wandb
import os
os.environ['WANDB_DISABLED'] = 'true'

# # Use wandb
# import wandb
# wandb.init(
#     project="aspect-based-term-extraction",
#     name="distilbert-base-uncased"
# )

In [19]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="abte-restaurants-distilbert-base-uncased",
    logging_dir="logs",
    learning_rate=2e-5,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    num_train_epochs=100,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="F1-score",
    # report_to="wandb",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_ds["train"],
    eval_dataset=preprocessed_ds["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1-score
1,0.042700,0.233329,0.806304
2,0.041200,0.239013,0.827382
3,0.027000,0.244601,0.827941
4,0.023600,0.248138,0.833107
5,0.025900,0.252910,0.834636
6,0.018300,0.259023,0.835850
7,0.015200,0.265793,0.834735
8,0.013900,0.274917,0.833231
9,0.012800,0.279838,0.841168
10,0.011000,0.301747,0.833437


Epoch,Training Loss,Validation Loss,F1-score
1,0.042700,0.233329,0.806304
2,0.041200,0.239013,0.827382
3,0.027000,0.244601,0.827941
4,0.023600,0.248138,0.833107
5,0.025900,0.252910,0.834636
6,0.018300,0.259023,0.835850
7,0.015200,0.265793,0.834735
8,0.013900,0.274917,0.833231
9,0.012800,0.279838,0.841168
10,0.011000,0.301747,0.833437


TrainOutput(global_step=1500, training_loss=0.0059710099250078206, metrics={'train_runtime': 3388.5731, 'train_samples_per_second': 106.298, 'train_steps_per_second': 0.443, 'total_flos': 8219543305550400.0, 'train_loss': 0.0059710099250078206, 'epoch': 100.0})

In [22]:
trainer.save_model("abte-restaurants-distilbert-base-uncased")
tokenizer.save_pretrained("abte-restaurants-distilbert-base-uncased")

('abte-restaurants-distilbert-base-uncased/tokenizer_config.json',
 'abte-restaurants-distilbert-base-uncased/special_tokens_map.json',
 'abte-restaurants-distilbert-base-uncased/vocab.txt',
 'abte-restaurants-distilbert-base-uncased/added_tokens.json',
 'abte-restaurants-distilbert-base-uncased/tokenizer.json')

## **Inference**

In [23]:
from transformers import pipeline

token_classifier = pipeline(
    "ner",
    model="abte-restaurants-distilbert-base-uncased",
    tokenizer="abte-restaurants-distilbert-base-uncased",
    aggregation_strategy="simple"
)

test_sentence = "The bred is top notch as well"
result = token_classifier(test_sentence)
result

Device set to use cuda:0


[{'entity_group': 'Term',
  'score': 0.5263697,
  'word': 'bred',
  'start': 4,
  'end': 8}]

In [ ]:
test_sentence = "The fish here is delicious"
result = token_classifier(test_sentence)
result

[{'entity_group': 'Term',
  'score': 0.88801813,
  'word': 'fish',
  'start': 4,
  'end': 8}]